### Лабораторная работа 6

6.1 Написать функции кодирования и декодирования для циклического кода (7, 4) с порождающим многочленом 𝑔(𝑥) = 1 + 𝑥^2 + 𝑥^3, исправляющего однократные ошибки и провести исследование этого кода для одно-, двух- и трёхкратных ошибок.

6.2 Написать функции кодирования и декодирования для циклического кода (15,9) с порождающим многочленом 𝑔(𝑥) = 1 + 𝑥^3 + 𝑥^4 + 𝑥^5 + 𝑥^6, исправляющего пакеты ошибок кратности 3 и провести исследование этого кода для пакетов ошибок длины 1, 2, 3 и 4.


Обратите внимание, что пакет ошибок длины t не означает, что все разряды в пределах этого пакета изменятся (см. лекции).

In [13]:
import numpy as np
import random

def divide_polynomials(dividend, divisor):
    current = list(dividend)
    divisor_length = len(divisor)

    while len(current) >= divisor_length:
        offset = len(current) - divisor_length
        for idx in range(divisor_length):
            current[offset + idx] ^= divisor[idx]
        while current and current[-1] == 0:
            current.pop()

    return np.array(current, dtype=int)

def multiply_polynomials(poly1, poly2):
    result_length = len(poly1) + len(poly2) - 1
    result = np.zeros(result_length, dtype=int)

    for i in range(len(poly1)):
        for j in range(len(poly2)):
            result[i + j] ^= poly1[i] & poly2[j]

    return result

def random_binary_sequence(length):
    return np.random.randint(0, 2, size=length, dtype=int)

def encode_message(data, generator):
    encoded = multiply_polynomials(data, generator)
    encoded %= 2
    return encoded

def introduce_errors(sequence, num_errors):
    corrupted = sequence.copy()
    error_positions = random.sample(range(len(sequence)), num_errors)

    for pos in error_positions:
        corrupted[pos] ^= 1

    print(f"Positions of errors: {error_positions}")
    return corrupted

def introduce_burst_errors(sequence, burst_size):
    errors = np.zeros(len(sequence), dtype=int)
    start = random.randint(0, len(sequence) - burst_size)

    for i in range(burst_size):
        if random.random() > 0.5:
            errors[start + i] = 1

    print(f"Burst error pattern: {errors}")
    return (sequence + errors) % 2

def decode_message(received, generator, max_correction, burst_mode=False):
    syndrome = divide_polynomials(received, generator)

    if not np.any(syndrome):
        return received

    for bit in range(len(received)):
        test_error = np.zeros(len(received), dtype=int)
        test_error[bit] = 1
        potential_correction = multiply_polynomials(syndrome, test_error) % 2

        if len(potential_correction) < len(received):
            potential_correction = np.pad(potential_correction, (len(received) - len(potential_correction), 0))
        elif len(potential_correction) > len(received):
            potential_correction = potential_correction[-len(received):]

        error_check = divide_polynomials(potential_correction, generator)

        if burst_mode:
            if len(error_check) <= max_correction and np.any(error_check):
                return (received + potential_correction) % 2
        else:
            if np.sum(error_check) <= max_correction:
                return (received + potential_correction) % 2

    print("Unable to correct the error.")
    return None

def simulate_experiment(generator, msg_length, max_correction, burst_mode=False, max_errors=3):
    message = random_binary_sequence(msg_length)
    print(f"Original message: {message}")
    encoded = encode_message(message, generator)
    print(f"Encoded message: {encoded}")

    for error_count in range(1, max_errors + 1):
        print(f"\n--- Errors: {error_count} ---")
        if burst_mode:
            received = introduce_burst_errors(encoded, error_count)
        else:
            received = introduce_errors(encoded, error_count)

        print(f"Received message: {received}")
        corrected = decode_message(received, generator, max_correction, burst_mode)
        print(f"Decoded message: {corrected}")

if __name__ == "__main__":
    print("Example 1: Cyclic code g(x) = 1 + x^2 + x^3, correcting single errors")
    gen1 = np.array([1, 0, 1, 1])
    simulate_experiment(gen1, msg_length=4, max_correction=1, max_errors=3)

    print("\nExample 2: Cyclic code (15,9) g(x) = 1 + x^3 + x^4 + x^5 + x^6, correcting burst errors of size 3")
    gen2 = np.array([1, 0, 0, 1, 1, 1, 1])
    simulate_experiment(gen2, msg_length=9, max_correction=3, burst_mode=True, max_errors=4)


Example 1: Cyclic code g(x) = 1 + x^2 + x^3, correcting single errors
Original message: [1 0 0 0]
Encoded message: [1 0 1 1 0 0 0]

--- Errors: 1 ---
Positions of errors: [4]
Received message: [1 0 1 1 1 0 0]
Decoded message: [0 0 1 1 1 0 0]

--- Errors: 2 ---
Positions of errors: [2, 3]
Received message: [1 0 0 0 0 0 0]
Decoded message: [0 0 0 0 0 0 0]

--- Errors: 3 ---
Positions of errors: [5, 1, 3]
Received message: [1 1 1 0 0 1 0]
Decoded message: [0 1 1 0 0 1 0]

Example 2: Cyclic code (15,9) g(x) = 1 + x^3 + x^4 + x^5 + x^6, correcting burst errors of size 3
Original message: [1 0 0 0 1 1 1 0 0]
Encoded message: [1 0 0 1 0 0 0 1 0 1 1 0 1 0 0]

--- Errors: 1 ---
Burst error pattern: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Received message: [1 0 0 1 0 0 0 1 0 1 1 0 1 0 0]
Decoded message: [1 0 0 1 0 0 0 1 0 1 1 0 1 0 0]

--- Errors: 2 ---
Burst error pattern: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Received message: [1 0 0 1 0 0 0 1 0 1 1 1 1 0 0]
Decoded message: [0 0 0 1 0 0 0 1 0 1 1 1 1 0 0]